# **SUPPLY CHAIN ANALYTICS USING *SUPABASE* AND *N8N***


## **1. Install Required Libaries To Connect with the Dataset On Cloud**

In [1]:
!pip install psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.0 MB/s eta 0:00:00


## **2. Dataset Integration from Cloud**

In [2]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="aws-1-ap-northeast-1.pooler.supabase.com",
    database="postgres",
    user="postgres.avwapozrubjtcwhdllzp",
    password="Sagar#gbs1",
    port="5432"
)

In [3]:
table_names = ["fact_order_line", "fact_aggregate", "dim_targets_orders","dim_products","dim_customers"]  # replace with your table names

dfs = {}  # dict to store DataFrames

for tbl in table_names:
    query = f'SELECT * FROM "{tbl}"'  # quotes handle lowercase/mixed-case names
    dfs[tbl] = pd.read_sql(query, conn)

# Example: see one table
dfs["fact_order_line"].head()

/tmp/ipython-input-1092671955.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[tbl] = pd.read_sql(query, conn)


,order_id,order_placement_date,customer_id,product_id,order_qty,agreed_delivery_date,actual_delivery_date,delivery_qty,In Full,On Time,On Time In Full
0,FMR34203601,2025-01-03,789203,25891601,110,04-03-2025,04-03-2025,110,1,1,1
1,FMR32320302,2025-01-03,789320,25891203,347,02-03-2025,02-03-2025,347,1,1,1
2,FMR33320501,2025-01-03,789320,25891203,187,03-03-2025,03-03-2025,150,0,1,0
3,FMR34220601,2025-01-03,789220,25891203,235,04-03-2025,04-03-2025,235,1,1,1
4,FMR33703603,2025-01-03,789703,25891203,176,03-03-2025,03-03-2025,176,1,1,1


In [4]:
dfs["fact_aggregate"].head()

,order_id,customer_id,order_placement_date,on_time,in_full,otif
0,FMR32103503,789103,2025-03-01,1,0,0
1,FMR34103403,789103,2025-03-02,1,0,0
2,FMR32103602,789103,2025-03-03,1,0,0
3,FMR33103602,789103,2025-03-04,1,0,0
4,FMR33103401,789103,2025-03-05,1,0,0


In [5]:
dfs["dim_targets_orders"].head()

,customer_id,ontime_target%,infull_target%,otif_target%
0,789201,87,81,70
1,789202,85,81,69
2,789203,92,76,70
3,789301,89,78,69
4,789303,88,78,69


In [6]:
dfs["dim_products"].head()

,product_name,product_id,category,price_INR,price_USD
0,AM Milk 500,25891101,Dairy,125,4.0
1,AM Milk 250,25891102,Dairy,62,2.0
2,AM Milk 100,25891103,Dairy,25,1.0
3,AM Butter 100,25891201,Dairy,60,2.0
4,AM Butter 250,25891202,Dairy,150,4.0


In [7]:
dfs["dim_customers"].head()

,customer_id,customer_name,city,currency
0,789101,Costco Wholesale,"New Jersey, US",USD
1,789102,Vijay Stores,Ahmedabad,INR
2,789103,Vijay Stores,Vadodara,INR
3,789121,Coolblue,Ahmedabad,INR
4,789122,Coolblue,Vadodara,INR


# **3. Calculate OTIF Performance**



**Reasoning**:
The subtask requires calculating OTIF performance metrics for each customer from two different dataframes, then merging these results with customer dimension data, and finally displaying the head and descriptive statistics of the combined performance. This involves several pandas operations including grouping, aggregation, and merging.



In [8]:
dfs["fact_aggregate"]['otif'] = pd.to_numeric(dfs["fact_aggregate"]['otif'], errors='coerce')
dfs["fact_order_line"]['On Time In Full'] = pd.to_numeric(dfs["fact_order_line"]['On Time In Full'], errors='coerce')

customer_otif_from_agg = dfs["fact_aggregate"].groupby('customer_id')['otif'].mean().reset_index()
customer_otif_from_agg['otif_percentage_agg'] = customer_otif_from_agg['otif'] * 100
customer_otif_from_agg = customer_otif_from_agg[['customer_id', 'otif_percentage_agg']]

customer_otif_from_line = dfs["fact_order_line"].groupby('customer_id')['On Time In Full'].mean().reset_index()
customer_otif_from_line['otif_percentage_line'] = customer_otif_from_line['On Time In Full'] * 100
customer_otif_from_line = customer_otif_from_line[['customer_id', 'otif_percentage_line']]

customer_performance = pd.merge(dfs["dim_customers"], customer_otif_from_agg, on='customer_id', how='left')
customer_performance = pd.merge(customer_performance, customer_otif_from_line, on='customer_id', how='left')

print("Customer Performance DataFrame Head:")
print(customer_performance.head())

print("\nDescriptive Statistics for OTIF Percentages:")
print(customer_performance[['otif_percentage_agg', 'otif_percentage_line']].describe())

Customer Performance DataFrame Head:
   customer_id     customer_name            city currency  \
0       789101  Costco Wholesale  New Jersey, US      USD   
1       789102      Vijay Stores       Ahmedabad      INR   
2       789103      Vijay Stores        Vadodara      INR   
3       789121          Coolblue       Ahmedabad      INR   
4       789122          Coolblue        Vadodara      INR   

   otif_percentage_agg  otif_percentage_line  
0            33.838384             58.914729  
1            35.751295             52.475248  
2             9.686610             25.531915  
3            20.982987             20.588235  
4             7.462687              4.587156  

Descriptive Statistics for OTIF Percentages:
       otif_percentage_agg  otif_percentage_line
count            35.000000             35.000000
mean             30.100808             47.169730
std              12.947645             22.687995
min               5.588822              1.709402
25%              19.910

# **4. Analyze Revenue and Quantity Sold**




**Reasoning**:
The first instruction is to merge the `fact_order_line` DataFrame with the `dim_products` DataFrame and then calculate 'total_revenue'. I will perform these steps within a single code block.



In [9]:
order_details = pd.merge(dfs["fact_order_line"], dfs["dim_products"], on='product_id', how='left')
order_details['total_revenue'] = order_details['order_qty'] * order_details['price_INR']

product_sales = order_details.groupby(['product_id', 'product_name']).agg(
    total_quantity_sold=('order_qty', 'sum'),
    total_product_revenue=('total_revenue', 'sum')
).reset_index()

print("Product Sales DataFrame Head:")
print(product_sales.head())

print("\nDescriptive Statistics for Product Sales:")
print(product_sales.describe())

customer_sales = order_details.groupby('customer_id').agg(
    total_quantity_ordered=('order_qty', 'sum'),
    total_customer_revenue=('total_revenue', 'sum')
).reset_index()

customer_sales_performance = pd.merge(customer_sales, dfs["dim_customers"], on='customer_id', how='left')

print("\nCustomer Sales Performance DataFrame Head:")
print(customer_sales_performance.head())

print("\nDescriptive Statistics for Customer Sales Performance:")
print(customer_sales_performance.describe())

Product Sales DataFrame Head:
   product_id   product_name  total_quantity_sold  total_product_revenue
0    25891101    AM Milk 500                87562               10945250
1    25891102    AM Milk 250                96008                5952496
2    25891103    AM Milk 100                78069                1951725
3    25891201  AM Butter 100                52774                3166440
4    25891202  AM Butter 250                66427                9964050

Descriptive Statistics for Product Sales:
         product_id  total_quantity_sold  total_product_revenue
count  1.800000e+01            18.000000           1.800000e+01
mean   2.589135e+07         50764.333333           5.559567e+06
std    1.757358e+02         28227.476579           5.481488e+06
min    2.589110e+07         12431.000000           6.642600e+05
25%    2.589120e+07         26671.000000           1.766010e+06
50%    2.589135e+07         47324.500000           3.417925e+06
75%    2.589150e+07         77355.000000 

# **5. Identify High-Value Products and Customers**

### Subtask:
Identify products and customers that contribute significantly to the business based on metrics like total order quantity and derived revenue, utilizing 'fact_order_line', 'dim_products', and 'dim_customers'.


**Reasoning**:
To identify high-value products, I will sort the `product_sales` DataFrame by 'total_product_revenue' and 'total_quantity_sold' in descending order and display the top 5. Similarly, for high-value customers, I will sort the `customer_sales_performance` DataFrame by 'total_customer_revenue' and 'total_quantity_ordered' in descending order and display the top 5.



In [10]:
top_products = product_sales.sort_values(by=['total_product_revenue', 'total_quantity_sold'], ascending=False).head(5)
print("\nTop 5 High-Value Products:")
print(top_products)

top_customers = customer_sales_performance.sort_values(by=['total_customer_revenue', 'total_quantity_ordered'], ascending=False).head(5)
print("\nTop 5 High-Value Customers:")
print(top_customers)


Top 5 High-Value Products:
    product_id     product_name  total_quantity_sold  total_product_revenue
5     25891203    AM Butter 500                75471               22641300
12    25891501  AM Biscuits 750                37180               11154000
0     25891101      AM Milk 500                87562               10945250
4     25891202    AM Butter 250                66427                9964050
13    25891502  AM Biscuits 500                41875                8375000

Top 5 High-Value Customers:
    customer_id  total_quantity_ordered  total_customer_revenue  \
5        789201                   31815                 3948019   
17       789420                   34726                 3829655   
14       789401                   30202                 3575655   
20       789501                   30011                 3572545   
0        789101                   28687                 3486188   

       customer_name            city currency  
5          Rel Fresh  New Jersey, US

# **6. Segment Customers**

### Subtask:
Segment customers based on their OTIF performance and profitability (or sales value), combining insights from previous analysis steps using 'fact_order_line', 'fact_aggregate', 'dim_customers', and 'dim_products'.


**Reasoning**:
To segment customers, I will first merge the `customer_performance` and `customer_sales_performance` DataFrames to consolidate all relevant metrics. Then, I will define profitability and OTIF performance segments using `pd.qcut` based on total customer revenue and OTIF percentage, respectively. Finally, I will combine these into a single segment column and display the results.



In [11]:
customer_segments_df = pd.merge(customer_performance, customer_sales_performance, on='customer_id', how='inner')

# Define profitability segments
customer_segments_df['profitability_segment'] = pd.qcut(customer_segments_df['total_customer_revenue'], q=3, labels=['Low Value', 'Medium Value', 'High Value'])

# Define OTIF performance segments
customer_segments_df['otif_segment'] = pd.qcut(customer_segments_df['otif_percentage_line'], q=3, labels=['Poor OTIF', 'Average OTIF', 'Good OTIF'])

# Create a combined segment column
customer_segments_df['combined_segment'] = customer_segments_df['profitability_segment'].astype(str) + ' - ' + customer_segments_df['otif_segment'].astype(str)

print("Customer Segments DataFrame Head:")
print(customer_segments_df.head())

print("\nDistribution of Combined Customer Segments:")
print(customer_segments_df['combined_segment'].value_counts())

Customer Segments DataFrame Head:
   customer_id   customer_name_x          city_x currency_x  \
0       789101  Costco Wholesale  New Jersey, US        USD   
1       789102      Vijay Stores       Ahmedabad        INR   
2       789103      Vijay Stores        Vadodara        INR   
3       789121          Coolblue       Ahmedabad        INR   
4       789122          Coolblue        Vadodara        INR   

   otif_percentage_agg  otif_percentage_line  total_quantity_ordered  \
0            33.838384             58.914729                   28687   
1            35.751295             52.475248                   23482   
2             9.686610             25.531915                   22898   
3            20.982987             20.588235                   22509   
4             7.462687              4.587156                   23490   

   total_customer_revenue   customer_name_y          city_y currency_y  \
0                 3486188  Costco Wholesale  New Jersey, US        USD   
1     

# **7. Visualize Key Performance Indicators**

### Subtask:
Generate interactive visualizations using Plotly to represent key performance indicators (KPIs) such as OTIF rates, sales trends, and customer segments. Ensure legends are provided for clarity.


**Reasoning**:
The subtask requires generating interactive visualizations using Plotly for OTIF rates, sales trends, and customer segments. The first step is to import the necessary library, `plotly.express`, to create these plots.



In [12]:
import plotly.express as px

**Reasoning**:
The next step is to create the bar chart for customer OTIF performance as requested in the subtask instructions. This involves using `plotly.express.bar` with the `customer_performance` DataFrame.



In [15]:
import plotly.graph_objects as go

# Scatter plot for Customer OTIF Performance
fig_otif = px.scatter(customer_performance, x='customer_id', y='otif_percentage_line',
                title='Customer OTIF Performance (from Order Line)',
                labels={'customer_id': 'Customer ID', 'otif_percentage_line': 'OTIF Percentage (from Order Line)'})

# Set x-axis type to 'category' to prevent numeric formatting (e.g., K for thousands)
fig_otif.update_xaxes(type='category')

# Add a line for the average OTIF percentage
average_otif = customer_performance['otif_percentage_line'].mean()
fig_otif.add_trace(go.Scatter(
    x=customer_performance['customer_id'],
    y=[average_otif] * len(customer_performance), # Repeat average value for all customers
    mode='lines',
    name=f'Average OTIF ({average_otif:.2f}%)',
    line=dict(color='red', dash='dash')
))
fig_otif.update_layout(showlegend=True)
fig_otif.show()

# Sort product_sales by total_product_revenue to get top 10 products
top_10_products = product_sales.sort_values(by='total_product_revenue', ascending=False).head(10)

fig_product_revenue = px.bar(top_10_products, x='product_name', y='total_product_revenue',
                           title='Top 10 Products by Revenue',
                           labels={'product_name': 'Product Name', 'total_product_revenue': 'Total Revenue (INR)'})
fig_product_revenue.show()

fig_treemap = px.treemap(customer_segments_df, path=['combined_segment', 'customer_name_x'],
                       values='total_customer_revenue',
                       title='Customer Segmentation by Revenue and OTIF')
fig_treemap.show()

# **8. Synthesize Actionable Insights**

### Subtask:
Based on all analyses and visualizations, formulate five concrete and actionable insights to enhance profitability and supply chain efficiency.


### **Actionable Insights:**

1.  **Prioritize Service for High-Value, Poor OTIF Customers:** Customers in the 'High Value - Poor OTIF' segment (e.g., customers like 'Best Market') are contributing significantly to revenue but are experiencing low On-Time In-Full performance. This indicates a high risk of churn for valuable clients. **Action**: Implement a dedicated customer service and logistics improvement program specifically for these high-value customers to address their delivery issues promptly and improve their OTIF rates.
2.  **Optimize Inventory and Logistics for Top-Revenue Products:** Products like 'AM Butter 500', 'AM Biscuits 750', and 'AM Milk 500' are consistently among the top revenue generators. Ensuring their availability and efficient delivery is crucial. **Action**: Conduct a detailed supply chain audit for these top products to identify bottlenecks, optimize inventory levels, and enhance delivery routes and processes to maintain high fulfillment rates.
3.  **Implement Targeted Interventions for Low OTIF Customers (Overall):** Many customers, particularly those in the 'Low Value - Poor OTIF' and 'Medium Value - Poor OTIF' segments, show consistently low OTIF percentages across both `fact_aggregate` and `fact_order_line` data. **Action**: For these customer groups, investigate the root causes of poor OTIF performance (e.g., order accuracy, transportation issues, warehouse efficiency). Consider implementing tiered service agreements or offering incentives for improved forecast accuracy to these customers.
4.  **Leverage "Good OTIF" Customers as Case Studies/Partners:** Customers in the 'High Value - Good OTIF' and 'Medium Value - Good OTIF' segments are performing well in both profitability and service. They represent successful partnerships. **Action**: Analyze the characteristics and operational alignment with these successful customers to understand best practices. Use these insights to develop strategies for improving relationships and performance with other customer segments, potentially through knowledge sharing or preferred partner programs.
5.  **Strategic Focus on Profitability and OTIF Alignment:** There are instances where customer OTIF performance from `fact_aggregate` and `fact_order_line` vary significantly, indicating potential data inconsistencies or different measurement scopes. **Action**: Standardize OTIF measurement across all data sources or provide clear explanations for discrepancies. For customers with high revenue but only average OTIF, explore opportunities to further enhance their service to solidify loyalty and prevent competitors from poaching them due to service gaps. For example, 'Costco Wholesale' is a high-value customer with average OTIF from order line, suggesting room for improvement to secure this relationship.

# **9. Generate Comprehensive Summary**

### Subtask:
Provide a comprehensive summary of the entire analysis, including the key findings, the five actionable insights, and recommendations for potential next steps.


## Summary:

### Data Analysis Key Findings

*   **OTIF Performance**:
    *   The average On-Time In-Full (OTIF) performance across customers was 30.10% (from `fact_aggregate`) and 47.17% (from `fact_order_line`), indicating a notable difference between the two data sources.
    *   Individual customer OTIF percentages ranged from 5.59% to 46.78% (`fact_aggregate`) and 1.71% to 73.79% (`fact_order_line`).
*   **Product Performance**:
    *   Products had `total_quantity_sold` ranging from 12,431 to 96,008 units and `total_product_revenue` from \$664,260 to \$22,641,300 across 18 unique products.
    *   The top 5 high-value products by revenue are: 'AM Butter 500' (\$22,641,300, 75,471 units), 'AM Biscuits 750' (\$11,154,000, 37,180 units), 'AM Milk 500' (\$10,945,250, 87,562 units), 'AM Butter 250' (\$9,964,050, 66,427 units), and 'AM Biscuits 500' (\$8,375,000, 41,875 units).
*   **Customer Performance**:
    *   Customers' `total_quantity_ordered` ranged from 20,384 to 34,726 units, and `total_customer_revenue` from \$2,010,075 to \$3,948,019 across 35 unique customers.
    *   The top 5 high-value customers by revenue are: 'Rel Fresh' (\$3,948,019, 31,815 units), 'Lidl' (\$3,829,655, 34,726 units), 'Wegmans' (\$3,575,655, 30,202 units), 'Best Market' (\$3,572,545, 30,011 units), and 'Costco Wholesale' (\$3,486,188, 28,687 units).
*   **Customer Segmentation**:
    *   Customers were segmented into 'Low Value', 'Medium Value', and 'High Value' based on `total_customer_revenue`, and 'Poor OTIF', 'Average OTIF', and 'Good OTIF' based on `otif_percentage_line`.
    *   The most frequent combined segments were 'Low Value - Poor OTIF' and 'High Value - Good OTIF', each with 6 customers.
*   **Visualizations**: Interactive bar charts displayed customer OTIF performance and top 10 products by revenue, while a treemap illustrated customer segmentation by revenue and OTIF.

### Insights or Next Steps

*   **Prioritize Service for High-Value, Poor OTIF Customers**: Implement dedicated improvement programs for customers like 'Best Market', who are high-value but have poor OTIF performance, to mitigate churn risk.
*   **Optimize Inventory and Logistics for Top-Revenue Products**: Conduct supply chain audits for 'AM Butter 500', 'AM Biscuits 750', and 'AM Milk 500' to ensure availability and efficient delivery.
*   **Implement Targeted Interventions for Low OTIF Customers**: Investigate root causes of poor OTIF for customers in 'Low Value - Poor OTIF' and 'Medium Value - Poor OTIF' segments and consider tiered service agreements or incentives.
*   **Leverage "Good OTIF" Customers as Case Studies**: Analyze best practices from 'High Value - Good OTIF' and 'Medium Value - Good OTIF' customers to improve relationships and performance with other segments.
*   **Standardize OTIF Measurement and Address Discrepancies**: Standardize OTIF measurement across data sources and explore opportunities to enhance service for high-revenue, average OTIF customers like 'Costco Wholesale' to solidify loyalty.
